In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import optuna
from sklearn.model_selection import TimeSeriesSplit
from catboost import CatBoostRegressor

In [ ]:
np.random.seed(41)

In [ ]:
wildfire_path = "/kaggle/input/forest-fire-prediction-epoch-hackathon/wildfire_sizes_before_2010.csv"
weather_path  = "/kaggle/input/marioo/merged_data.csv"
state_path    = "/kaggle/input/forest-fire-prediction-epoch-hackathon/merged_state_data.csv"

wildfire_df = pd.read_csv(wildfire_path)
weather_df  = pd.read_csv(weather_path)
state_df    = pd.read_csv(state_path)

In [ ]:
# Rename columns for consistency
weather_df.rename(columns={'year_month': 'month', 'State': 'STATE'}, inplace=True)
state_df.rename(columns={'State': 'STATE'}, inplace=True)

# Clean percentage columns (strip '%' if present and convert to float)
state_df['Percentage of Federal Land'] = state_df['Percentage of Federal Land'].apply(lambda x: float(str(x).rstrip('%')))
state_df['Urbanization Rate (%)'] = state_df['Urbanization Rate (%)'].apply(lambda x: float(str(x).rstrip('%')))

# Convert month columns to datetime
wildfire_df['month'] = pd.to_datetime(wildfire_df['month'], format='%Y-%m')
weather_df['month']  = pd.to_datetime(weather_df['month'], format='%Y-%m')

In [ ]:
# Merge wildfire sizes with weather and state data
train_df = pd.merge(wildfire_df, weather_df, on=['STATE', 'month'], how='left')
train_df = pd.merge(train_df, state_df, on='STATE', how='left')

# Complete the time series: every state for every month 1992-01 to 2010-12
all_states = state_df['STATE'].unique()
train_months = pd.date_range(start='1992-01-01', end='2010-12-01', freq='MS')  # MS = Month Start
full_index = pd.MultiIndex.from_product([all_states, train_months], names=['STATE', 'month'])
full_train_df = pd.DataFrame(index=full_index).reset_index()

# Merge with existing training data (missing rows will have NaN)
train_df = pd.merge(full_train_df, train_df, on=['STATE', 'month'], how='left')

# For missing wildfire data, assume 0 fire size
train_df['total_fire_size'] = train_df['total_fire_size'].fillna(0)

# Fill missing weather features (if any) with median
for feat in ['PRCP','EVAP','TMIN','TMAX']:
    if feat in train_df.columns:
        train_df[feat] = train_df[feat].fillna(train_df[feat].median())

In [ ]:
# Date features
train_df['year'] = train_df['month'].dt.year
train_df['month_num'] = train_df['month'].dt.month
train_df['month_sin'] = np.sin(2 * np.pi * train_df['month_num'] / 12)
train_df['month_cos'] = np.cos(2 * np.pi * train_df['month_num'] / 12)
train_df['year_since_1992'] = train_df['year'] - 1992

# Advanced weather lag features for selected columns
weather_cols = ['PRCP', 'TMAX']
for col in weather_cols:
    train_df.sort_values(['STATE', 'month'], inplace=True)
    train_df[f'{col}_lag1'] = train_df.groupby('STATE')[col].shift(1)
    train_df[f'{col}_lag2'] = train_df.groupby('STATE')[col].shift(2)
    train_df[f'{col}_lag3'] = train_df.groupby('STATE')[col].shift(3)
    train_df[f'{col}_roll3'] = train_df.groupby('STATE')[col].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
    train_df[f'{col}_roll6'] = train_df.groupby('STATE')[col].rolling(window=6, min_periods=1).mean().reset_index(0, drop=True)

# Interaction feature: temperature spread and its rolling average
train_df['temp_spread'] = train_df['TMAX'] - train_df['TMIN']
train_df['temp_spread_roll3'] = train_df.groupby('STATE')['temp_spread'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)

# Compute historical (median) fire size per state and month from training data
hist_fire = train_df.groupby(['STATE', 'month_num'])['total_fire_size'].median().reset_index()
hist_fire.rename(columns={'total_fire_size': 'hist_fire_size'}, inplace=True)
hist_fire['log_hist_fire_size'] = np.log1p(hist_fire['hist_fire_size'])
train_df = pd.merge(train_df, hist_fire[['STATE', 'month_num', 'log_hist_fire_size']], on=['STATE','month_num'], how='left')

# Compute historical weather medians per state and month (from training period)
weather_hist = train_df.groupby(['STATE', 'month_num'])[['PRCP','TMAX','TMIN']].median().reset_index()
weather_hist.rename(columns={'PRCP':'hist_PRCP','TMAX':'hist_TMAX','TMIN':'hist_TMIN'}, inplace=True)
train_df = pd.merge(train_df, weather_hist, on=['STATE','month_num'], how='left')

# Create weather anomaly features
train_df['prcp_anomaly'] = train_df['PRCP'] - train_df['hist_PRCP']
train_df['tmax_anomaly'] = train_df['TMAX'] - train_df['hist_TMAX']
train_df['tmin_anomaly'] = train_df['TMIN'] - train_df['hist_TMIN']

# Log-transform target variable
train_df['log_fire_size'] = np.log1p(train_df['total_fire_size'])

# One-hot encode STATE (for state-specific effects)
state_dummies = pd.get_dummies(train_df['STATE'], prefix='STATE')
train_df = pd.concat([train_df, state_dummies], axis=1)

In [ ]:
feature_cols = [
    'year', 
    'month_num', 
    'month_sin', 
    'month_cos', 
    'year_since_1992',
    'PRCP', 
    'PRCP_lag1', 'PRCP_lag2', 'PRCP_lag3', 'PRCP_roll3', 'PRCP_roll6',
    'EVAP', 
    'TMIN', 
    'TMAX', 
    'TMAX_lag1', 'TMAX_lag2', 'TMAX_lag3', 'TMAX_roll3', 'TMAX_roll6',
    'temp_spread', 'temp_spread_roll3',
    'prcp_anomaly', 'tmax_anomaly', 'tmin_anomaly',
    'mean_elevation', 
    'Land Area (sq mi)', 
    'Water Area (sq mi)', 
    'Total Area (sq mi)', 
    'Percentage of Federal Land', 
    'Urbanization Rate (%)',
    'log_hist_fire_size',
    'hist_PRCP', 'hist_TMAX', 'hist_TMIN',
    'Value'
]
state_dummy_cols = [col for col in train_df.columns if col.startswith("STATE_")]
feature_cols.extend(state_dummy_cols)

# Fill missing feature values with median
train_df[feature_cols] = train_df[feature_cols].fillna(train_df[feature_cols].median())

In [ ]:
# Use data before 2010 for training and data in 2010 for validation.
train_data = train_df[train_df['year'] < 2010].copy()
val_data   = train_df[train_df['year'] == 2010].copy()

X_train = train_data[feature_cols]
y_train = train_data['log_fire_size']
X_val   = val_data[feature_cols]
y_val   = val_data['log_fire_size']

In [ ]:
def objective(trial):
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 3, 10),
        'verbose': -1,
        'seed': 41
    }
    tscv = TimeSeriesSplit(n_splits=3)
    cv_scores = []
    for train_idx, valid_idx in tscv.split(X_train):
        X_tr, X_val_cv = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_tr, y_val_cv = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        lgb_train_cv = lgb.Dataset(X_tr, label=y_tr)
        lgb_val_cv = lgb.Dataset(X_val_cv, label=y_val_cv)
        model_cv = lgb.train(param, lgb_train_cv, num_boost_round=2000,
                             valid_sets=[lgb_val_cv],
                             callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)])
        preds = model_cv.predict(X_val_cv, num_iteration=model_cv.best_iteration)
        rmse = np.sqrt(np.mean((np.expm1(preds) - np.expm1(y_val_cv)) ** 2))
        cv_scores.append(rmse)
    return np.mean(cv_scores)

print("Starting hyperparameter tuning for LightGBM...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
best_params = study.best_trial.params
print("Best parameters found:", best_params)

# Update LightGBM parameters with tuned values
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': best_params['learning_rate'],
    'num_leaves': best_params['num_leaves'],
    'min_data_in_leaf': best_params['min_data_in_leaf'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    'bagging_freq': best_params['bagging_freq'],
    'verbose': -1,
    'seed': 41
}

In [ ]:
# --- LightGBM ---
lgb_train_data = lgb.Dataset(X_train, label=y_train)
lgb_val_data   = lgb.Dataset(X_val, label=y_val, reference=lgb_train_data)
print("Training final LightGBM model...")
lgb_model = lgb.train(
    lgb_params,
    lgb_train_data,
    num_boost_round=2500,
    valid_sets=[lgb_train_data, lgb_val_data],
    valid_names=['train', 'val'],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(100)]
)

# --- XGBoost ---
xgb_params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.01,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 41
}
dtrain = xgb.DMatrix(X_train, label=y_train)
dval   = xgb.DMatrix(X_val, label=y_val)
print("Training final XGBoost model...")
xgb_model = xgb.train(
    xgb_params,
    dtrain,
    num_boost_round=2500,
    evals=[(dtrain, 'train'), (dval, 'val')],
    early_stopping_rounds=100,
    verbose_eval=100
)

# --- CatBoost ---
cat_model = CatBoostRegressor(
    iterations=2500,
    learning_rate=0.01,
    depth=6,
    loss_function='RMSE',
    random_seed=41,
    early_stopping_rounds=100,
    verbose=100
)
print("Training final CatBoost model...")
cat_model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

In [ ]:
lgb_pred_log = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
xgb_pred_log = xgb_model.predict(dval, iteration_range=(0, xgb_model.best_iteration))
cat_pred_log = cat_model.predict(X_val)

# Ensemble (average predictions in log-space)
ensemble_pred_log = (lgb_pred_log + xgb_pred_log + cat_pred_log) / 3.0

val_pred = np.expm1(ensemble_pred_log)
val_true = np.expm1(y_val)
log_errors = np.abs(np.log((val_pred + 1e-9) / (val_true + 1e-9)))
log_errors_clipped = np.minimum(log_errors, 10)
val_score = log_errors_clipped.mean()
print("Validation Ensemble Log-Error Score:", val_score)

In [ ]:
# Complete time series for test: every state for every month from 2011-01 to 2015-12.
test_months = pd.date_range(start='2011-01-01', end='2015-12-01', freq='MS')
full_test_index = pd.MultiIndex.from_product([all_states, test_months], names=['STATE', 'month'])
full_test_df = pd.DataFrame(index=full_test_index).reset_index()

# Merge with weather and state data
test_df = pd.merge(full_test_df, weather_df, on=['STATE', 'month'], how='left')
test_df = pd.merge(test_df, state_df, on='STATE', how='left')

# Create date features for test set
test_df['year'] = test_df['month'].dt.year
test_df['month_num'] = test_df['month'].dt.month
test_df['month_sin'] = np.sin(2 * np.pi * test_df['month_num'] / 12)
test_df['month_cos'] = np.cos(2 * np.pi * test_df['month_num'] / 12)
test_df['year_since_1992'] = test_df['year'] - 1992

for col in weather_cols:
    test_df.sort_values(['STATE', 'month'], inplace=True)
    test_df[f'{col}_lag1'] = test_df.groupby('STATE')[col].shift(1)
    test_df[f'{col}_lag2'] = test_df.groupby('STATE')[col].shift(2)
    test_df[f'{col}_lag3'] = test_df.groupby('STATE')[col].shift(3)
    test_df[f'{col}_roll3'] = test_df.groupby('STATE')[col].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)
    test_df[f'{col}_roll6'] = test_df.groupby('STATE')[col].rolling(window=6, min_periods=1).mean().reset_index(0, drop=True)

# Interaction features for test data
test_df['temp_spread'] = test_df['TMAX'] - test_df['TMIN']
test_df['temp_spread_roll3'] = test_df.groupby('STATE')['temp_spread'].rolling(window=3, min_periods=1).mean().reset_index(0, drop=True)

# Merge historical features from training data into test set
test_df = pd.merge(test_df, hist_fire[['STATE','month_num','log_hist_fire_size']], on=['STATE','month_num'], how='left')
test_df = pd.merge(test_df, weather_hist, on=['STATE','month_num'], how='left')

# Create weather anomaly features
test_df['prcp_anomaly'] = test_df['PRCP'] - test_df['hist_PRCP']
test_df['tmax_anomaly'] = test_df['TMAX'] - test_df['hist_TMAX']
test_df['tmin_anomaly'] = test_df['TMIN'] - test_df['hist_TMIN']

# One-hot encode STATE for test data
state_dummies_test = pd.get_dummies(test_df['STATE'], prefix='STATE')
test_df = pd.concat([test_df, state_dummies_test], axis=1)

for col in feature_cols:
    if col not in test_df.columns:
        test_df[col] = 0
test_df[feature_cols] = test_df[feature_cols].fillna(test_df[feature_cols].median())

In [ ]:
dtest = xgb.DMatrix(test_df[feature_cols])
lgb_test_pred_log = lgb_model.predict(test_df[feature_cols], num_iteration=lgb_model.best_iteration)
xgb_test_pred_log = xgb_model.predict(dtest, iteration_range=(0, xgb_model.best_iteration))
cat_test_pred_log = cat_model.predict(test_df[feature_cols])
ensemble_test_pred_log = (lgb_test_pred_log + xgb_test_pred_log + cat_test_pred_log) / 3.0
test_df['total_fire_size'] = np.expm1(ensemble_test_pred_log)
test_df.loc[test_df['total_fire_size'] < 0, 'total_fire_size'] = 0

In [ ]:
test_df = test_df.sort_values(['STATE', 'month']).reset_index(drop=True)
test_df['ID'] = test_df.index
test_df['month'] = test_df['month'].dt.strftime('%Y-%m')
submission = test_df[['ID', 'STATE', 'month', 'total_fire_size']].copy()
submission.to_csv('submission.csv', index=False)
print("Final submission file saved as 'submission.csv'!")